In [1]:
# importing required libraries
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv('/content/drive/MyDrive/balanced_data.csv')

In [4]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense,Dropout,LSTM,Embedding,SimpleRNN, GRU,Activation,Flatten
from tensorflow.keras.layers import Conv1D,MaxPool1D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,CSVLogger
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import sequence
from sklearn.metrics import (precision_score, recall_score,confusion_matrix,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
import pandas as pd
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import matplotlib.pyplot as plt

In [5]:
import tensorflow.keras
print('keras: %s' % tensorflow.keras.__version__)

keras: 2.11.0


In [6]:
data.drop(data.columns[0] , inplace=True , axis=1)

## Add Unknown Label

In [7]:
data.Label.value_counts()

BENIGN              1296087
DoS Hulk             231073
PortScan             158930
DDoS                 128027
DoS GoldenEye         10293
FTP-Patator            7938
SSH-Patator            5897
DoS slowloris          5796
DoS Slowhttptest       5499
Bot                    1966
Infiltration             36
Heartbleed               11
Name: Label, dtype: int64

In [8]:
data.Label.unique()

array(['BENIGN', 'Infiltration', 'DDoS', 'DoS slowloris',
       'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'Heartbleed',
       'PortScan', 'Bot', 'FTP-Patator', 'SSH-Patator'], dtype=object)

In [9]:
UnknownLabel = ['DoS GoldenEye' , 'FTP-Patator' , 'SSH-Patator' , 'DoS slowloris' , 'DoS Slowhttptest' , 'Bot' ,'Infiltration' , 'Heartbleed']

In [10]:
data.Label.replace(to_replace=UnknownLabel , value = 'Unknown',inplace=True)

In [11]:
data.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,35929,2100,443,2744,49916,6,14653,65,3,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,195043,2210,52673,2608,53,17,160,91765,2,2,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,58449,1337,45245,1593,53,17,27,295,2,2,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,70012,696,58208,865,53,17,80,53034,4,2,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,31918,2777,443,1590,50506,6,119,71,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [12]:
data.Label.value_counts()

BENIGN      1296087
DoS Hulk     231073
PortScan     158930
DDoS         128027
Unknown       37436
Name: Label, dtype: int64

# **Data Normalization**

In [13]:
# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [14]:
# selecting numeric attributes columns from data
numeric_col = data.select_dtypes(include='number').columns

In [15]:
# using standard scaler for normalizing
std_scaler = StandardScaler()
def normalization(df,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
  return df

In [16]:
# calling the normalization() function
data = normalization(data.copy(),numeric_col)

In [17]:
# data after normalization
data.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,-0.646799,0.324411,-1.948333,0.204423,2.428566,-0.662668,2.051947,-0.472445,-0.008076,-0.009102,...,0.002719,-0.128601,-0.097503,-0.146881,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,BENIGN
1,1.783219,0.430907,0.508829,0.114303,-0.421518,1.505849,-0.394452,-0.469837,-0.009562,-0.007983,...,0.002719,-0.128601,-0.097503,-0.146881,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,BENIGN
2,-0.302869,-0.414284,0.159378,-0.558287,-0.421518,1.505849,-0.416902,-0.472439,-0.009562,-0.007983,...,0.002735,-0.128601,-0.097503,-0.146881,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,BENIGN
3,-0.126277,-1.034865,0.769223,-1.040696,-0.421518,1.505849,-0.407956,-0.470938,-0.006590,-0.007983,...,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,BENIGN
4,-0.708056,0.979845,-1.948333,-0.560275,2.462290,-0.662668,-0.401372,-0.472445,-0.011049,-0.009102,...,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,BENIGN


# **One-hot-encoding**

In [18]:
# selecting numeric attributes columns from data
non_numeric_col = data.select_dtypes(exclude='number').columns

In [19]:
# creating a dataframe with only categorical attributes
categorical = data[non_numeric_col]
categorical.head()

,Label
0,BENIGN
1,BENIGN
2,BENIGN
3,BENIGN
4,BENIGN


In [20]:
# one-hot-encoding categorical attributes using pandas.get_dummies() function
categorical = pd.get_dummies(categorical,columns=non_numeric_col)
categorical.head()

,Label_BENIGN,Label_DDoS,Label_DoS Hulk,Label_PortScan,Label_Unknown
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


# **Concatenate with original data :-)**

In [21]:
final_concatenated = pd.concat([ data ,categorical] , axis=1)

In [22]:
final_concatenated.drop(['Label'],axis=1 , inplace=True)

In [23]:
final_concatenated.to_csv('/content/drive/MyDrive/unknown_labelled_data.csv')